# Task
Analyze the "Churn Modelling Dataset" to predict customer churn for a bank. The analysis should include data cleaning, categorical feature encoding, training a classification model, and analyzing feature importance to identify key drivers of churn.

## Load the data

### Subtask:
Load the dataset from the provided CSV file into a pandas DataFrame.


**Reasoning**:
Import pandas and load the dataset into a DataFrame, then display the first 5 rows.



In [1]:
import pandas as pd

df = pd.read_csv('/content/Churn_Modelling.csv')
display(df.head())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data preprocessing

### Subtask:
Clean the data, handle missing values, encode categorical features, and split the data into training and testing sets.


**Reasoning**:
Drop irrelevant columns, check for missing values, perform one-hot encoding on categorical features, separate features and target, and split the data into training and testing sets.



In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Drop irrelevant columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Check for missing values
print("Missing values before handling:")
print(df.isnull().sum())

# Identify categorical features
categorical_features = ['Geography', 'Gender']

# Create a column transformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_features)],
    remainder='passthrough'
)

# Separate features (X) and target (y)
X = df.drop('Exited', axis=1)
y = df['Exited']

# Create a pipeline that first preprocesses and then potentially adds a model (not done here)
# This is useful for keeping preprocessing consistent between training and testing
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Apply the preprocessing to the features
X_processed = pipeline.fit_transform(X)

# Convert the processed features back to a DataFrame (optional, but useful for inspection)
# Get the new column names after one-hot encoding
new_column_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
remaining_columns = [col for col in X.columns if col not in categorical_features]
all_processed_columns = list(new_column_names) + remaining_columns

# Handle potential sparse matrix output from OneHotEncoder
if isinstance(X_processed, np.ndarray):
    X_processed_df = pd.DataFrame(X_processed, columns=all_processed_columns)
else:
    X_processed_df = pd.DataFrame(X_processed.toarray(), columns=all_processed_columns)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed_df, y, test_size=0.2, random_state=42)

print("\nShape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Missing values before handling:
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Shape of X_train: (8000, 13)
Shape of X_test: (2000, 13)
Shape of y_train: (8000,)
Shape of y_test: (2000,)


## Model training

### Subtask:
Train a classification model (e.g., Logistic Regression, Random Forest, or Gradient Boosting) on the training data.


**Reasoning**:
Train a Logistic Regression model on the training data.



In [3]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

## Model evaluation

### Subtask:
Evaluate the trained model's performance on the testing data using appropriate metrics (e.g., accuracy, precision, recall, F1-score).


**Reasoning**:
Calculate and print the evaluation metrics for the trained model on the test set.



In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.8000
Precision: 0.4521
Recall: 0.0840
F1-score: 0.1416


## Feature importance analysis

### Subtask:
Analyze feature importance to understand which features contribute most to customer churn.


**Reasoning**:
Access the coefficients of the trained Logistic Regression model, create a pandas Series from the coefficients, sort the feature importance in descending order, and print the sorted feature importance.



In [5]:
# Access the coefficients of the trained Logistic Regression model
coefficients = model.coef_[0]

# Create a pandas Series from the coefficients, using the column names of X_train as the index
feature_importance = pd.Series(coefficients, index=X_train.columns)

# Sort the feature importance in descending order
sorted_feature_importance = feature_importance.sort_values(ascending=False)

# Print or display the sorted feature importance
display(sorted_feature_importance)

,0
Age,0.047243
Geography_Germany,0.030440
Gender_Female,0.025407
Balance,0.000004
EstimatedSalary,-0.000001
CreditScore,-0.004454
HasCrCard,-0.011023
Geography_Spain,-0.012361
NumOfProducts,-0.027195
Geography_France,-0.032154


## Summarize findings

### Subtask:
Present the model's performance and feature importance insights.


**Reasoning**:
Summarize the model performance metrics and feature importance, and provide an overall summary of the findings.



In [6]:
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

print("\nKey Performance Metrics Explained:")
print("Accuracy: The proportion of correctly predicted cases (both churn and non-churn) out of the total number of cases.")
print("Precision: The proportion of correctly predicted churn cases out of all cases predicted as churn. A high precision means fewer false positives.")
print("Recall: The proportion of correctly predicted churn cases out of all actual churn cases. A high recall means fewer false negatives.")
print("F1-score: The harmonic mean of precision and recall, providing a balanced measure of the model's performance, especially useful in the presence of class imbalance.")

print("\nTop Features by Importance (Logistic Regression Coefficients):")
display(sorted_feature_importance)

print("\nDiscussion of Feature Impact:")
print("Based on the Logistic Regression coefficients:")
print(f"- Age ({sorted_feature_importance['Age']:.4f}) has the most significant positive impact on churn, meaning older customers are more likely to churn.")
print(f"- Geography_Germany ({sorted_feature_importance['Geography_Germany']:.4f}) also has a positive impact, suggesting customers in Germany are more likely to churn compared to the reference geography.")
print(f"- Gender_Female ({sorted_feature_importance['Gender_Female']:.4f}) has a positive impact, indicating female customers are slightly more likely to churn compared to male customers.")
print(f"- Tenure ({sorted_feature_importance['Tenure']:.4f}) has the most significant negative impact, suggesting customers with longer tenure are less likely to churn.")
print(f"- IsActiveMember ({sorted_feature_importance['IsActiveMember']:.4f}) also has a negative impact, indicating active members are less likely to churn.")

print("\nOverall Summary:")
print("The Logistic Regression model achieved an accuracy of {:.4f}, but the precision ({:.4f}) and recall ({:.4f}) for churn prediction are low, resulting in a low F1-score ({:.4f}). This suggests the model struggles to effectively identify customers who will churn, likely due to class imbalance in the dataset. The feature importance analysis indicates that Age is the strongest positive predictor of churn, while Tenure and being an Active Member are the strongest negative predictors. Geography_Germany and Gender_Female also show a positive association with churn. To improve churn prediction, further model tuning, exploring different classification algorithms, or addressing the class imbalance would be beneficial.".format(accuracy, precision, recall, f1))

Model Performance Metrics:
Accuracy: 0.8000
Precision: 0.4521
Recall: 0.0840
F1-score: 0.1416

Key Performance Metrics Explained:
Accuracy: The proportion of correctly predicted cases (both churn and non-churn) out of the total number of cases.
Precision: The proportion of correctly predicted churn cases out of all cases predicted as churn. A high precision means fewer false positives.
Recall: The proportion of correctly predicted churn cases out of all actual churn cases. A high recall means fewer false negatives.
F1-score: The harmonic mean of precision and recall, providing a balanced measure of the model's performance, especially useful in the presence of class imbalance.

Top Features by Importance (Logistic Regression Coefficients):


,0
Age,0.047243
Geography_Germany,0.030440
Gender_Female,0.025407
Balance,0.000004
EstimatedSalary,-0.000001
CreditScore,-0.004454
HasCrCard,-0.011023
Geography_Spain,-0.012361
NumOfProducts,-0.027195
Geography_France,-0.032154



Discussion of Feature Impact:
Based on the Logistic Regression coefficients:
- Age (0.0472) has the most significant positive impact on churn, meaning older customers are more likely to churn.
- Geography_Germany (0.0304) also has a positive impact, suggesting customers in Germany are more likely to churn compared to the reference geography.
- Gender_Female (0.0254) has a positive impact, indicating female customers are slightly more likely to churn compared to male customers.
- Tenure (-0.0889) has the most significant negative impact, suggesting customers with longer tenure are less likely to churn.
- IsActiveMember (-0.0591) also has a negative impact, indicating active members are less likely to churn.

Overall Summary:
The Logistic Regression model achieved an accuracy of 0.8000, but the precision (0.4521) and recall (0.0840) for churn prediction are low, resulting in a low F1-score (0.1416). This suggests the model struggles to effectively identify customers who will churn, like

## Summary:

### Data Analysis Key Findings

*   The dataset did not contain any missing values.
*   The data was successfully split into training and testing sets, with the training set containing 8000 samples and the testing set containing 2000 samples.
*   A Logistic Regression model was trained to predict churn.
*   The model achieved an accuracy of 0.8000 on the test set.
*   The precision for predicting churn was 0.4521, while the recall was significantly lower at 0.0840. The F1-score was 0.1416.
*   Feature importance analysis using the Logistic Regression coefficients indicated that Age has the most significant positive impact on churn (coefficient: 0.0472).
*   Geography\_Germany (coefficient: 0.0304) and Gender\_Female (coefficient: 0.0254) also showed positive associations with churn.
*   Tenure had the most significant negative impact on churn (coefficient: -0.0889), followed by IsActiveMember (coefficient: -0.0591).

### Insights or Next Steps

*   The model's low recall and F1-score for churn prediction suggest that it is not effectively identifying customers who will churn. This is likely due to the class imbalance in the dataset.
*   Future steps should focus on improving the model's ability to predict the minority class (churn). This could involve techniques such as:
    *   Addressing class imbalance (e.g., using oversampling, undersampling, or SMOTE).
    *   Exploring different classification algorithms that are less sensitive to class imbalance (e.g., Random Forest, Gradient Boosting, or models with built-in handling for imbalanced data).
    *   Tuning the hyperparameters of the chosen model.
    *   Investigating other features or creating new features that might be more predictive of churn.
